In [22]:
import re
from dataclasses import dataclass
from enum import Enum, auto
from queue import Queue

In [23]:
class Tag(Enum):
    ERROR = auto()
    TERM = auto()
    NTERM = auto()
    AXIOM = auto()
    COMMENT = auto()
    OPEN = auto()
    CLOSE = auto()
    UNMATCHED = auto()
    END = auto()


@dataclass
class Token:
    """Represents tokens for lexical analysis."""

    tag: Tag   
    value: str = ""


@dataclass
class CoordsToken(Token):
    start_idx: int = 0
    end_idx: int = 0

    def __repr__(self):
        return f"{self.tag} ({self.start_idx}, {self.end_idx}): {self.value}"

In [24]:
class Lexer:
    """Performs lexical analysis of input data."""

    def __init__(self, config: dict):
        self.config = config
        self._re_mapping = {
            r"'.*\n": Tag.COMMENT,
            r"<axiom <({})>>\n".format(config["nonterminal"]): Tag.AXIOM,
            config["open"]: Tag.OPEN,
            config["close"]: Tag.CLOSE,
            config["terminal"]: Tag.TERM,
            config["nonterminal"]: Tag.NTERM,   
        }

    def _match_token(self, input_str: str) -> Token:
        for pattern, tag in self._re_mapping.items():
            matched = re.match(pattern, input_str)
            if matched:
                return Token(tag=tag, value=matched.group())

        return Token(tag=Tag.UNMATCHED, value=input_str[0])

    def tokenize(self, input_str: str) -> Queue:
        tokens = Queue()
        idx = 0

        while idx < len(input_str):
            token = self._match_token(input_str[idx:])
            if token.tag == Tag.UNMATCHED and token.value.isspace() or token.tag == Tag.COMMENT:
                idx += len(token.value)
            else:
                if token.tag == Tag.AXIOM:
                    axiom_value = re.search(self.config["nonterminal"], token.value).group(0) # type: ignore
                    tokens.put(CoordsToken(Tag.AXIOM, axiom_value, idx, idx + len(token.value)))
                else:
                    tokens.put(CoordsToken(token.tag, token.value, idx, idx + len(token.value)))
                idx += len(token.value)

        tokens.put(CoordsToken(Tag.END, "", idx+1, idx+1))
        return tokens

In [25]:
config = {
    "open": "<",
    "close": ">",
    "terminal": r"[a-z\(\)\+\*]",
    "nonterminal": r"[A-Z]'?"
}

In [26]:
class Nonterminal:
    def __init__(self, symbol, rule_id=0):
        self.symbol = symbol
        self.rule_id = rule_id
        self.children = []

    def __eq__(self, other):
        return (isinstance(other, Nonterminal) and
                self.symbol == other.symbol)

    def __hash__(self):
        return hash(self.symbol)

    def __str__(self):
        return self.symbol
    
    def print(self, indent):
        print(indent + str(self) + ":")
        for child in self.children:
            child.print(indent + "\t")


class Terminal:
    def __init__(self, symbol):
        self.symbol = symbol

    def __eq__(self, other):
        return (isinstance(other, Terminal) and
                self.symbol == other.symbol)

    def __hash__(self):
        return hash(self.symbol)

    def __str__(self):
        return self.symbol
    
    def print(self, indent):
        print(indent + str(self))

In [27]:
class Table:
    def __init__(self):
        self.rule_ids = iter(range(100))
        self.mapping = {
            (Nonterminal("Grammar"), Tag.AXIOM): ([Terminal("Axiom"), Nonterminal("Rules")], next(self.rule_ids)),
            (Nonterminal("Rules"), Tag.OPEN): ([Nonterminal("Rule"), Nonterminal("RestRules")], next(self.rule_ids)),
            (Nonterminal("RestRules"), Tag.OPEN): ([Nonterminal("Rule"), Nonterminal("RestRules")], next(self.rule_ids)),
            (Nonterminal("RestRules"), Tag.END): ([], next(self.rule_ids)),
            (Nonterminal("Rule"), Tag.OPEN): ([Terminal("Open"), Terminal("Nterm"), Nonterminal("RHS"), Terminal("Close")], next(self.rule_ids)),
            (Nonterminal("RHS"), Tag.OPEN): ([Nonterminal("RHSTerm"), Nonterminal("RestRHS")], next(self.rule_ids)),
            (Nonterminal("RestRHS"), Tag.OPEN): ([Nonterminal("RHSTerm"), Nonterminal("RestRHS")], next(self.rule_ids)),
            (Nonterminal("RestRHS"), Tag.CLOSE): ([], next(self.rule_ids)),
            (Nonterminal("RestRHS"), Tag.END): ([], next(self.rule_ids)),
            (Nonterminal("RHSTerm"), Tag.OPEN): ([Terminal("Open"), Nonterminal("RHSFactor"), Nonterminal("RestRHSTerm"), Terminal("Close")], next(self.rule_ids)),
            (Nonterminal("RestRHSTerm"), Tag.TERM): ([Nonterminal("RHSFactor"), Nonterminal("RestRHSTerm")], next(self.rule_ids)),
            (Nonterminal("RestRHSTerm"), Tag.NTERM): ([Nonterminal("RHSFactor"), Nonterminal("RestRHSTerm")], next(self.rule_ids)),
            (Nonterminal("RestRHSTerm"), Tag.CLOSE): ([], next(self.rule_ids)),
            (Nonterminal("RestRHSTerm"), Tag.END): ([], next(self.rule_ids)),
            (Nonterminal("RHSFactor"), Tag.TERM): ([Terminal("Term")], next(self.rule_ids)),
            (Nonterminal("RHSFactor"), Tag.NTERM): ([Terminal("Nterm")], next(self.rule_ids)),
            (Nonterminal("RHSFactor"), Tag.CLOSE): ([], next(self.rule_ids)),
        }

In [28]:
class Leaf:
    def __init__(self, token):
        self.token = token
      
    def print(self, indent=""):
        print(f"{indent}Leaf: {self.token}")

class Inner:
    def __init__(self, nterm, rule_id):
        self.nterm = nterm
        self.rule_id = rule_id
        self.children = []
      
    def print(self, indent=""):
        print(f"{indent}Inner Node: {self.nterm}, rule: {self.rule_id}:")
        for child in self.children:
            child.print(indent + "\t")


def top_down(tokens):
    type_mapping = {
        Tag.AXIOM: "Axiom",
        Tag.TERM: "Term",
        Tag.NTERM: "Nterm",
        Tag.OPEN: "Open",
        Tag.CLOSE: "Close"
    }

    delta = Table()
    sparent = Inner(None, None)
    stack = [(sparent, Terminal('$')), (sparent, Nonterminal('Grammar'))]

    token = tokens.get()
    parent, X = stack.pop()
    
    while X.symbol != '$':
        if isinstance(X, Terminal):
            if X.symbol == type_mapping[token.tag]:
                parent.children.append(Leaf(token))
                token = tokens.get()
            else:
                raise ValueError(f"T. Ожидался {X}, Получен {token}")
        elif (X, token.tag) in delta.mapping.keys():
            inner = Inner(X, delta.mapping[X, token.tag][1])
            parent.children.append(inner)
            for elem in delta.mapping[X, token.tag][0][::-1]:
                stack.append((inner, elem))
        else:
            raise ValueError(f"Ожидался {X}, Получен {token}")
        
        parent, X = stack.pop()

    return sparent.children[0]

In [29]:
lexer = Lexer(config)

test_str = """' аксиома
<axiom <E>>
' правила грамматики
<E  <T E'>>
' и это комментарий
<E' <+ T E'> <>> 
<T  <F T'>>
<T' <* F T'> 'и это комментарий

 <>>
<F  <n> <( E )>>"""

tokens = lexer.tokenize(test_str)

In [30]:
v = top_down(tokens)

In [31]:
v.print()

Inner Node: Grammar, rule: 0:
	Leaf: Tag.AXIOM (10, 22): E
	Inner Node: Rules, rule: 1:
		Inner Node: Rule, rule: 4:
			Leaf: Tag.OPEN (43, 44): <
			Leaf: Tag.NTERM (44, 45): E
			Inner Node: RHS, rule: 5:
				Inner Node: RHSTerm, rule: 9:
					Leaf: Tag.OPEN (47, 48): <
					Inner Node: RHSFactor, rule: 15:
						Leaf: Tag.NTERM (48, 49): T
					Inner Node: RestRHSTerm, rule: 11:
						Inner Node: RHSFactor, rule: 15:
							Leaf: Tag.NTERM (50, 52): E'
						Inner Node: RestRHSTerm, rule: 12:
					Leaf: Tag.CLOSE (52, 53): >
				Inner Node: RestRHS, rule: 7:
			Leaf: Tag.CLOSE (53, 54): >
		Inner Node: RestRules, rule: 2:
			Inner Node: Rule, rule: 4:
				Leaf: Tag.OPEN (75, 76): <
				Leaf: Tag.NTERM (76, 78): E'
				Inner Node: RHS, rule: 5:
					Inner Node: RHSTerm, rule: 9:
						Leaf: Tag.OPEN (79, 80): <
						Inner Node: RHSFactor, rule: 14:
							Leaf: Tag.TERM (80, 81): +
						Inner Node: RestRHSTerm, rule: 11:
							Inner Node: RHSFactor, rule: 15:
								Leaf: Tag.NTE